In [1]:
# Third Party Imports
import pandas as pd
import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Local imports
import bat
from bat import log_to_dataframe
from bat import dataframe_to_matrix
print('DONE')

DONE


In [2]:
# Create a Pandas dataframe from a Bro log
bro_df = log_to_dataframe.LogToDataFrame('/home/dane/bro_logs/dns.log')

# Print out the head of the dataframe
bro_df.head()
print('DONE')

Successfully monitoring /home/dane/giant/dns.log...
DONE


In [3]:
# Using Pandas we can easily and efficiently compute additional data metrics
# Here we use the vectorized operations of Pandas/Numpy to compute query length
bro_df['query_length'] = bro_df['query'].str.len()
print('DONE')

DONE


## DNS records are a mix of numeric and categorical data
When we look at the dns records some of the data is numerical and some of it is categorical so we'll need a way of handling both data types in a generalized way. bat has a DataFrameToMatrix class that handles a lot of the details and mechanics of combining numerical and categorical data, we'll use below.

In [4]:
# These are the features we want (note some of these are categorical :)
features = ['AA', 'RA', 'RD', 'TC', 'Z', 'rejected', 'proto', 'query', 
            'qclass_name', 'qtype_name', 'rcode_name', 'query_length']
feature_df = bro_df[features]
feature_df.head()
print('DONE')

DONE


<div style="float: right; margin: -10px 40px -10px 40px"><img src="images/transformers.png" width="200px"></div>
## Transformers
**We'll now use a bat scikit-learn tranformer class to convert the Pandas DataFrame to a numpy ndarray (matrix). Yes it's awesome... I'm not sure it's Optimus Prime awesome.. but it's still pretty nice.**

In [ ]:
# Use the bat DataframeToMatrix class (handles categorical data)
# You can see below it uses a heuristic to detect category data. When doing
# this for real we should explicitly convert before sending to the transformer.
to_matrix = dataframe_to_matrix.DataFrameToMatrix()
bro_matrix = to_matrix.fit_transform(feature_df)
print('DONE')

Changing column proto to category...
Changing column qclass_name to category...
Changing column qtype_name to category...
Changing column rcode_name to category...
Normalizing column Z...
Normalizing column query_length...
DONE


In [ ]:
# Just showing that the class is tracking categoricals and normalization maps
print(to_matrix.cat_columns)
print(to_matrix.norm_map)
print('DONE')

['proto', 'qclass_name', 'qtype_name', 'rcode_name']
{'Z': (0, 1), 'query_length': (1, 88)}
DONE


In [ ]:
# Now we're ready for scikit-learn!
# Just some simple stuff for this example, KMeans and TSNE projection
kmeans = KMeans(n_clusters=5).fit_predict(bro_matrix)
projection = TSNE().fit_transform(bro_matrix)
print('DONE')

In [ ]:
# Now we can put our ML results back onto our dataframe!
bro_df['tsne_0'] = projection[:, 0] # Projection X Column
bro_df['tsne_1'] = projection[:, 1] # Projection Y Column
bro_df['cluster'] = kmeans
bro_df[['query', 'proto', 'tsne_0', 'tsne_1', 'cluster']].head()  # Showing the scikit-learn results in our dataframe
# Plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12.0
plt.rcParams['figure.figsize'] = 15.0, 7.0
# Now use dataframe group by cluster
cluster_groups = bro_df.groupby('cluster')

# Plot the Machine Learning results
fig, ax = plt.subplots()
colors = {0:'green', 1:'blue', 2:'red', 3:'orange', 4:'purple',}
for key, group in cluster_groups:
    group.plot(ax=ax, kind='scatter', x='tsne_0', y='tsne_1', alpha=0.01, s=2000,
               label='Cluster: {:d}'.format(key), color=colors[key])
print('DONE')

In [ ]:
# Now print out the details for each cluster
pd.set_option('display.width', 1000)
show_fields = ['query', 'Z', 'proto', 'qtype_name', 
               'cluster','AA', 'RA', 'RD',
               'TC', 'rejected','qclass_name',
               'rcode_name', 'query_length']
for key, group in cluster_groups:
    print('\nCluster {:d}: {:d} observations'.format(key, len(group)))
    print(group[show_fields].head())
print(len(bro_df))
# Cluster 0 = normal query
# Cluster 1 = successful dns query to the "internet" by an HP laserjet printer
# Cluster 2 = blank query without query type and length of 1
# Cluster 3 = normal successful dns query to the "internet"
# Cluster 4 = variable query_length, successful dns query of type PTR - Reverse-lookup Pointer records (PTR Record)—allows a DNS resolver to provide an IP address and receive a hostname (reverse DNS lookup).
print('DONE')

In [ ]:
for key, group in cluster_groups:
    print('\nCluster {:d}: {:d} observations'.format(key, len(group)))
    print(group[show_fields])
print('DONE')